# STEP 0 - Preprocessing of the DataSet

Here, the dataset is analyzed and afterwards preprocessed.

## Imports

In [34]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

cincinnati = create_engine('sqlite:///cincinnati.db')

## Data Analysis

First, we read the dataset into a database in order to get a general understanding. We have the columns:
* ASSET : Unique indentifier for each vehicle equipped with GPS
* ASSETNHOOD: The neighborhood the GPS ping was located in
* DISTANCE_TRAVELED: Distance travelled since last GPS ping
* HEADING: Direction the vehicle is moveing at the time of the GPS ping
* ID_HAM_PVMNT_PLYGN: The unique identifier for the street segment the vehicle was on at the time of the GPS ping
* LATITUDE
* LOADTS: Time that the data was loaded from the zonar GPS system
* LONGITUDE
* ODOMETER: The total distance traveled in miles by the vehicle over its lifetime at the time of the GPS ping.
* PPOLYLABEL: The name of the street the GPS ping was on
* REASONS: A coded value for the action or actions that the vehcile is completing at the time of the GPS ping
* REASONS_TEXT: The decoded action or actions that the vehicle is completing at the time of the GPS ping
* SPEED: Speed of the vehicle at the time of the GPS ping in miles per hour
* STREETFROM: The beginning cross street of the street segment that the GPS ping was on
* STREETTO: The end cross street of the street segment that the GPS ping was on
* TIME

In [35]:
df = pd.read_csv("./cincinnati/Year_to_Date_Vehicle_GPS_Data__Department_of_Public_Services.csv", sep=",",)
print(df.head(10))

df.to_sql('cincinnati', con=cincinnati, if_exists="replace")

   ASSET       ASSETNHOOD  DISTANCE_TRAVELED HEADING ID_HAM_PVMNT_PLYGN  \
0    317  WEST PRICE HILL                0.1     NNE         S-HAM03339   
1    483   OVER-THE-RHINE                0.0       S         S-HAM20247   
2    130  WEST PRICE HILL                0.1       N         S-HAM03328   
3    276    PADDOCK HILLS                0.6       S         S-HAM30071   
4    317              NaN                0.5     SSE                NaN   
5    197  WEST PRICE HILL                0.4       E         S-HAM01835   
6    380              NaN                0.6       W                NaN   
7    318              NaN                0.6     SWW         S-HAM52807   
8     72       CORRYVILLE                1.3       W       I-GJ15223474   
9    301              NaN                0.2       N                NaN   

    LATITUDE        LOADTS  LONGITUDE  ODOMETER  \
0  39.108567  2.021010e+13 -84.595120   38216.1   
1  39.114371  2.021010e+13 -84.512336    8378.6   
2  39.108329  2.02101

In [36]:
res=pd.read_sql_query('SELECT count(*) FROM cincinnati', cincinnati)
print(res)

   count(*)
0  14502805


There are 14.502.805 rows in the dataset.

In [37]:
res=pd.read_sql_query('SELECT ASSET, "count" FROM ' +
            '(SELECT count(ASSET) as "count", ASSET FROM cincinnati GROUP BY ASSET)' +
            'ORDER BY "count" DESC LIMIT 10', cincinnati)

print(res)

   ASSET   count
0    153  165652
1    445  141781
2    450  140814
3    261  138953
4    255  138790
5    487  135507
6    228  134339
7    449  131684
8    210  130801
9    300  125974


Drop all columns that will not be used. (Heading could be interesting for prediction)

In [38]:
df.drop(['ASSETNHOOD', 'DISTANCE_TRAVELED', 'HEADING', 'ID_HAM_PVMNT_PLYGN', 'LOADTS', 'ODOMETER', 'PPOLYLABEL', 'REASONS', 'REASONS_TEXT', 'SPEED', 'STREETFROM', 'STREETTO'], axis=1, inplace=True)

Use the 'reverse_geocoder' as before to map the geo coordinates to areas.

In [39]:
import reverse_geocoder as rg

coords = np.vstack((df["LONGITUDE"].values, df["LATITUDE"].values)).T
coord_array = [(i[1], i[0]) for i in coords]

coord_array

[(39.108567, -84.59512),
 (39.114371000000006, -84.512336),
 (39.108329, -84.585651),
 (39.169484999999995, -84.478541),
 (39.27178, -84.597099),
 (39.12342, -84.597555),
 (39.142382, -84.534018),
 (39.229744000000004, -84.636625),
 (39.135496999999994, -84.51025600000001),
 (39.271931, -84.597054),
 (39.127959999999995, -84.584665),
 (39.122991, -84.50836),
 (39.150419, -84.54406),
 (39.105145, -84.594658),
 (39.142476, -84.532933),
 (39.23418, -84.628637),
 (39.103159000000005, -84.590361),
 (39.141309, -84.472409),
 (39.160609, -84.420905),
 (39.123843, -84.60295),
 (39.144659000000004, -84.432757),
 (39.151258, -84.586937),
 (39.202774, -84.54405899999999),
 (39.185371, -84.60499899999999),
 (39.132536, -84.432958),
 (39.148995, -84.496184),
 (39.113723, -84.607545),
 (39.168849, -84.46704799999999),
 (39.126248, -84.55030699999999),
 (39.196773, -84.547777),
 (39.214459999999995, -84.68633299999999),
 (39.104631, -84.507524),
 (39.120092, -84.57498000000001),
 (39.130328000000006,

In [40]:
coord_array = [(i[1], i[0]) for i in coords]
search = rg.search(coord_array)
search

[OrderedDict([('lat', '39.12117'),
              ('lon', '-84.60633'),
              ('name', 'Covedale'),
              ('admin1', 'Ohio'),
              ('admin2', 'Hamilton County'),
              ('cc', 'US')]),
 OrderedDict([('lat', '39.09145'),
              ('lon', '-84.49578'),
              ('name', 'Newport'),
              ('admin1', 'Kentucky'),
              ('admin2', 'Campbell County'),
              ('cc', 'US')]),
 OrderedDict([('lat', '39.12117'),
              ('lon', '-84.60633'),
              ('name', 'Covedale'),
              ('admin1', 'Ohio'),
              ('admin2', 'Hamilton County'),
              ('cc', 'US')]),
 OrderedDict([('lat', '39.18728'),
              ('lon', '-84.488'),
              ('name', 'Elmwood Place'),
              ('admin1', 'Ohio'),
              ('admin2', 'Hamilton County'),
              ('cc', 'US')]),
 OrderedDict([('lat', '39.25283'),
              ('lon', '-84.59245'),
              ('name', 'Northgate'),
              ('admin1

In [41]:
values= []
for i in range(len(search)):
    values.append(search[i])

coord_search_df = pd.DataFrame(values)
coord_search_df

,lat,lon,name,admin1,admin2,cc
0,39.12117,-84.60633,Covedale,Ohio,Hamilton County,US
1,39.09145,-84.49578,Newport,Kentucky,Campbell County,US
2,39.12117,-84.60633,Covedale,Ohio,Hamilton County,US
3,39.18728,-84.488,Elmwood Place,Ohio,Hamilton County,US
4,39.25283,-84.59245,Northgate,Ohio,Hamilton County,US
...,...,...,...,...,...,...
14502800,39.09256,-84.54744,Ludlow,Kentucky,Kenton County,US
14502801,39.167,-84.49855,Saint Bernard,Ohio,Hamilton County,US
14502802,39.167,-84.49855,Saint Bernard,Ohio,Hamilton County,US
14502803,39.167,-84.49855,Saint Bernard,Ohio,Hamilton County,US


In [42]:
print("Name : ", coord_search_df.name.nunique())
print("Admin1 : ", coord_search_df.admin1.nunique())
print("Admin2 : ", coord_search_df.admin2.nunique())

Name :  166
Admin1 :  6
Admin2 :  26


Now, the dataset is joined with the areas.

In [43]:
zones = df.copy()
zones['area_name'] = coord_search_df['name']
zones

,ASSET,LATITUDE,LONGITUDE,TIME,area_name
0,317,39.108567,-84.595120,2.021010e+13,Covedale
1,483,39.114371,-84.512336,2.021010e+13,Newport
2,130,39.108329,-84.585651,2.021010e+13,Covedale
3,276,39.169485,-84.478541,2.021010e+13,Elmwood Place
4,317,39.271780,-84.597099,2.021010e+13,Northgate
...,...,...,...,...,...
14502800,446,39.137762,-84.535466,2.021010e+13,Ludlow
14502801,228,39.157666,-84.547908,2.021010e+13,Saint Bernard
14502802,446,39.144670,-84.525625,2.021010e+13,Saint Bernard
14502803,258,39.139572,-84.533399,2.021010e+13,Saint Bernard


In [44]:
count = zones.area_name.value_counts()
count

Saint Bernard           3107116
Ludlow                  1895889
Covedale                1190999
Fairfax                  934287
Norwood                  681201
                         ...   
Edgewood                      2
Osgood                        2
Saint Peter                   2
Nantucket                     1
Country Squire Lakes          1
Name: area_name, Length: 166, dtype: int64

The dataset got so many records, so it is fine to drop the least used zones.
Only the 100 most used zones are kept

In [45]:
idx = count.loc[count.index[:100]].index
zones = zones.loc[zones.area_name.isin(idx)]

In [46]:
print("Name : ", zones.area_name.nunique())
print("Row Count : ", len(zones))

Name :  100
Row Count :  14500904


Previously: 14.502.805
Now: 14.500.904
Approximately 2000 rows were lost by dropping all the records with the 61 least used areas.

In [47]:
#disable warning
pd.options.mode.chained_assignment = None
zones['location_id'] = zones.groupby('area_name', sort=False).ngroup()
zones.head(10)

,ASSET,LATITUDE,LONGITUDE,TIME,area_name,location_id
0,317,39.108567,-84.595120,2.021010e+13,Covedale,0
1,483,39.114371,-84.512336,2.021010e+13,Newport,1
2,130,39.108329,-84.585651,2.021010e+13,Covedale,0
3,276,39.169485,-84.478541,2.021010e+13,Elmwood Place,2
4,317,39.271780,-84.597099,2.021010e+13,Northgate,3
5,197,39.123420,-84.597555,2.021010e+13,Covedale,0
6,380,39.142382,-84.534018,2.021010e+13,Saint Bernard,4
7,318,39.229744,-84.636625,2.021010e+13,Dry Ridge,5
8,72,39.135497,-84.510256,2.021010e+13,Saint Bernard,4
9,301,39.271931,-84.597054,2.021010e+13,Northgate,3


In [48]:
# convert the 'Date' column to datetime format
# 20210104085517 -> 2021-01-04 08:55.17
# 20210104131316 -> 2021-01-04 13:13.16
zones['TIME'] = zones['TIME'].astype(str).str.slice(0,13)
zones['time']= pd.to_datetime(zones['TIME'], format='%Y%m%d%H%M%S')
zones.head(10)

,ASSET,LATITUDE,LONGITUDE,TIME,area_name,location_id,time
0,317,39.108567,-84.595120,2021010408551,Covedale,0,2021-01-04 08:55:01
1,483,39.114371,-84.512336,2021010409313,Newport,1,2021-01-04 09:31:03
2,130,39.108329,-84.585651,2021010413131,Covedale,0,2021-01-04 13:13:01
3,276,39.169485,-84.478541,2021010411241,Elmwood Place,2,2021-01-04 11:24:01
4,317,39.271780,-84.597099,2021010413225,Northgate,3,2021-01-04 13:22:05
5,197,39.123420,-84.597555,2021010409362,Covedale,0,2021-01-04 09:36:02
6,380,39.142382,-84.534018,2021010401444,Saint Bernard,4,2021-01-04 01:44:04
7,318,39.229744,-84.636625,2021010413100,Dry Ridge,5,2021-01-04 13:10:00
8,72,39.135497,-84.510256,2021010408292,Saint Bernard,4,2021-01-04 08:29:02
9,301,39.271931,-84.597054,2021010412103,Northgate,3,2021-01-04 12:10:03


In [49]:
count_nan = zones['time'].isnull().sum() + zones['location_id'].isnull().sum() + zones['ASSET'].isnull().sum()

# printing the number of values present in the columns time, location_id and ASSET
print('Number of NaN values present: ' + str(count_nan))

Number of NaN values present: 0


In [50]:
def is_weekend(week_day):
    return week_day > 4

The data is split in temporal components and a sin-/cos-transformation is executed.

In [51]:
zones["month"] = zones.time.dt.month
zones["day"] = zones.time.dt.day
zones["clock"] = zones.time.dt.hour + zones.time.dt.minute/60 + zones.time.dt.second/3600
zones["week_day"] = zones.time.dt.dayofweek
zones["is_weekend"] = zones.apply(lambda x: is_weekend(x["week_day"]), axis=1)

In [52]:
zones['clock_sin'] = np.sin(2 * np.pi * zones['clock']/24.0)
zones['clock_cos'] = np.cos(2 * np.pi * zones['clock']/24.0)
zones['day_sin'] = np.sin(2 * np.pi * zones['day']/30.0)
zones['day_cos'] = np.cos(2 * np.pi * zones['day']/30.0)
zones['month_sin'] = np.sin(2 * np.pi * zones['month']/12.0)
zones['month_cos'] = np.cos(2 * np.pi * zones['month']/12.0)
zones['week_day_sin'] = np.sin(2 * np.pi * zones['week_day']/7.0)
zones['week_day_cos'] = np.cos(2 * np.pi * zones['week_day']/7.0)

Now, the columns that are not needed any more will be dropped.

In [53]:
zones.drop(['LATITUDE', 'LONGITUDE', 'TIME', 'time', 'month', 'day', 'clock', 'week_day', 'area_name'], axis=1, inplace=True)

In [54]:
def move_first(df, column_name):
    fc = df.pop(column_name)
    df.insert(0, column_name, fc)

In [55]:
move_first(zones, 'location_id')

In [56]:
zones.rename(columns={'ASSET':'vehicle_id'}, inplace=True)

In [57]:
zones.head(10)

,location_id,vehicle_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
0,0,317,False,0.722314,-0.691566,0.743145,0.669131,0.5,0.866025,0.0,1.0
1,1,483,False,0.605120,-0.796134,0.743145,0.669131,0.5,0.866025,0.0,1.0
2,0,130,False,-0.313233,-0.949676,0.743145,0.669131,0.5,0.866025,0.0,1.0
3,2,276,False,0.156363,-0.987700,0.743145,0.669131,0.5,0.866025,0.0,1.0
4,3,317,False,-0.350548,-0.936545,0.743145,0.669131,0.5,0.866025,0.0,1.0
5,0,197,False,0.587668,-0.809102,0.743145,0.669131,0.5,0.866025,0.0,1.0
6,4,380,False,0.438633,0.898666,0.743145,0.669131,0.5,0.866025,0.0,1.0
7,5,318,False,-0.300706,-0.953717,0.743145,0.669131,0.5,0.866025,0.0,1.0
8,4,72,False,0.795914,-0.605410,0.743145,0.669131,0.5,0.866025,0.0,1.0
9,3,301,False,-0.043837,-0.999039,0.743145,0.669131,0.5,0.866025,0.0,1.0


In [58]:
zones.to_csv('././cincinnati/cincinatti_zones.csv', index=False)

In [59]:
single_vehicle = pd.DataFrame(zones.loc[zones.vehicle_id == 153])

In [60]:
single_vehicle.head(10)

,location_id,vehicle_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
235,4,153,False,-0.725424,-0.688302,0.743145,0.669131,0.5,0.866025,0.0,1.0
257,10,153,False,-0.619379,-0.785092,0.743145,0.669131,0.5,0.866025,0.0,1.0
273,4,153,False,0.378379,-0.925651,0.743145,0.669131,0.5,0.866025,0.0,1.0
356,4,153,False,0.635798,-0.771856,0.743145,0.669131,0.5,0.866025,0.0,1.0
430,11,153,False,-0.492424,-0.870356,0.743145,0.669131,0.5,0.866025,0.0,1.0
567,4,153,False,0.476903,-0.878956,0.743145,0.669131,0.5,0.866025,0.0,1.0
573,18,153,False,-0.697999,-0.716099,0.743145,0.669131,0.5,0.866025,0.0,1.0
737,10,153,False,0.121725,-0.992564,0.743145,0.669131,0.5,0.866025,0.0,1.0
771,10,153,False,0.004363,-0.999990,0.743145,0.669131,0.5,0.866025,0.0,1.0
834,13,153,False,0.618923,-0.785452,0.743145,0.669131,0.5,0.866025,0.0,1.0


In [61]:
single_vehicle.to_csv('././cincinnati/cincinatti_zones_153.csv', index=False)